In [ ]:
!pip install tensorflow-text

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download CellPhones -f CellPhonesRating.csv

In [3]:
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow.keras import datasets, layers, models

In [15]:
df = pd.read_csv("/content/CellPhonesRating.csv.zip")
df = df.dropna(how='any',axis=0)
df = df.drop_duplicates(subset =['productID', 'reviewerID'] , keep = 'first')

In [18]:
df = df[:300]

In [ ]:
df.drop('reviewText',axis='columns', inplace=True)

In [ ]:
df[(df['reviewerID'] == 'AB6CHQWHZW4TV') & (df['productID'] == '7508492919')]['rating'].size

In [22]:
user_item ={}
for item in df['productID']:
  user_item[item] = {}
  for user in df['reviewerID'].unique():
      user_item[item][user] = 0

In [23]:
user_item_df = pd.DataFrame(user_item)

In [ ]:
user_item_df

In [32]:
df.values[1]

array([1, 5.0, 'A269FLZCB4GIPV', '7508492919'], dtype=object)

In [ ]:
for row in df.values:
  user_item_df[row[2],row[3]] = row[1]

In [ ]:
user_item_df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
#Train/Test data split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'],df['rating'], stratify=df['rating'])

In [ ]:
encoder_input = layers.Input(shape=(),name='user_item')
